In [ ]:

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [33]:
import os
import time
from datetime import date
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException


In [48]:
from bs4 import BeautifulSoup
import requests

# url="https://www.amazon.in/Ainclu-Womens-Classic-Layered-Lace-Up/dp/B00K69XHGY/ref=b2b_gw_d_csf_b2b_sccl_1/258-8837396-8497212?pd_rd_w=SvU1u&content-id=amzn1.sym.80118820-a13a-4810-a5d3-0816775952fa&pf_rd_p=80118820-a13a-4810-a5d3-0816775952fa&pf_rd_r=80CBD46BM18R81225Y26&pd_rd_wg=0hL82&pd_rd_r=c516329a-4a79-48da-8432-a3ec36ed9d0f&pd_rd_i=B083QDVK83&psc=1"
# result=requests.get(url)
# doc=BeautifulSoup(result.text,"html.parser")
# prices=doc.find_all(text="₹")
# parent=prices[0].parent
# print(parent)

# #tags=doc.find_all(["p","div",";li"])

# #tags=doc.find_all(text=re.compile("\$"),limit=1)
# #for tag in tags:
#    #print(tag.strip())
from requests_html import HTMLSession
import json
import time
#Python is an “object-oriented programming language.” This means that almost all the code is 
#implemented using a special construct called classes. Programmers use classes to keep related 
#things together. This is done using the keyword “class,” which is a grouping of object-oriented 
#constructs.
#A class is a code template for creating objects. 
#Objects have member variables and have behaviour associated with them.
#An object is created using the constructor of the class. 
#This object will then be called the instance of the class. In Python we create instances in the following manner
#Instance = class(arguments)
class Reviews:
    def __init__(self) -> None:
        self.session=HTMLSession() #this is our object tht we ll use whenever class instance is called
        self.headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
        #self.url= f'https://www.amazon.in/Rovars-U2301-Boys-Black-Light/product-reviews/{self.asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
        self.url=None
        #self.driver = None
        self.product_name = None
        self.formatted_product_name = None
        self.product_url=None
        self.product_page_url=None
       
        # to get to amazon india website
    # def open_amazon_website(self):
    #     opt=Options()# creates instance for options
    #     opt.add_argument("--disable-infobars")
    #     opt.add_argument("--disable-extensions")
    #     opt.add_argument('--log-level=OFF')
    #     opt.add_experimental_option('excludeSwitches', ['enable-logging'])
    #     url = "https://www.amazon.in/"
    #     self.driver = webdriver.Chrome(ChromeDriverManager(driver_version='114.0.5735.90').install())
    #     # go to that page using driver
    #     self.driver.get(url)
    #     #to load the website
    #     time.sleep(3)
     #https://www.flipkart.com/search?q=PS5+Digital+Standalone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&as-pos=1&as-type=HISTORY   
            
        #creating url to get to product page eg mobile phones,playstation
    def open_all_product_page(self):
        s=input("\n>>Enter the product name")
        self.product_name=s
        self.formatted_product_name=self.product_name.replace(" ","+")
        
        #fits product name in url
        self.product_url = "https://www.flipkart.com/search?q={}&otracker"
        self.product_url = self.product_url.format(self.formatted_product_name)
        
        print(">> Product URL: ", self.product_url)
        

            
        #to get links of all products in the page
    
    def get_product_link(self):
        # product_name=input('\n>>Enter product name')
        # self.product_name=product_name
        # self.formatted_product_name=self.product_name.replace(" ","-")
        result=requests.get(self.product_url,headers=self.headers)
        soup=BeautifulSoup(result.content,"html.parser")
       
        #links=soup.find_all("a",attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" })
        #links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
        
        links=soup.find_all("a",attrs={'rel':'noopener norefferer'})
        print(links)
        self.product_page_url=links[0]
       
        #print(">>Product page URL:",self.product_page_url)    
                
    def get_review_url(self):
        self.url=self.product_page_url.replace("dp","product-reviews")+"/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
        
    #all reviews from all review pages of one product
    def pagination(self,page):
        r=self.session.get(self.url+str(page))
        if not r.html.find('div[data-hook=review]'):
            return False
        else:
            return r.html.find('div[data-hook=review]')
          

   # review parsing
    def parse(self,reviews):
        total=[]  
        rating_count=0   
        ratings=0  
        avg_rating=0 
        total_sentimental_score=0
        for review in reviews:
            title=review.find('a[data-hook=review-title]',first=True).text
            rating=review.find('i[data-hook=review-star-rating] span',first=True).text
            body=review.find('span[data-hook=review-body] span',first=True).text.replace('\n','').strip()
            rating_count=rating_count+1
            ratings=ratings+(int(rating[0]))
            sentiment= sentiment_pipeline(body)[0]
            if sentiment['label']=='POSITIVE':
               total_sentimental_score=sentiment['score']+total_sentimental_score
            else:
               total_sentimental_score=total_sentimental_score-sentiment['score']
            
            data={
                'rating_count':rating_count,
                'title':title,
                'rating':rating,
                'body':body[:100],
                'sentiment':sentiment
            }
            total.append(data)
        avg_rating=ratings/rating_count
        avg_sentimental_score=total_sentimental_score/rating_count
        return total,avg_rating,avg_sentimental_score
    
    
    
    def save(self,results):
        with open(self.product_name+'reviews.json','w') as f:
            json.dump(results,f)
#https://www.amazon.in/Sony-PS5-PlayStation-Console/dp/B0BRCP72X8

if __name__=='__main__':
    amz=Reviews()

    amz.open_all_product_page()
    
    amz.get_product_link()
    amz.get_review_url()
    results=[]
    average_rating=0
    for i in range(1,5):
        
        time.sleep(0.3)
        reviews=amz.pagination(i)
        if reviews is not False:
            results.append(amz.parse(reviews))
            temp,average_rating,avg_sentimental_score=amz.parse(reviews)
            
        else :
            print("No more pages")
            break
    print(f"The average rating is: {average_rating}\n")
    print(f"The average sentimental score is: {avg_sentimental_score}")
    if avg_sentimental_score>0:
        print("Overall sentimental review is POSITIVE")
    else:
        print("Overall sentimental review is NEGATIVE")
    
    amz.save(results) 
            



>> Product URL:  https://www.flipkart.com/search?q=GADGETSWRAP+PS5-Skn-6+Playstation+PS5+Digital+Edition+Charred+Robot+3M+Skin+Gaming+Accessory+Kit++(Black,+For+PS5)&otracker
[]


IndexError: list index out of range